# Using a Local LLM in DSPy

### Configure llama3.1 as LM using Ollama

In [ ]:
import dspy

lm = dspy.OllamaLocal(model='llama3.1')
dspy.settings.configure(lm=lm)

## Zero-shot QA

In [16]:
# This is the signature for the predictor. It is a simple question and answer model.
class BasicQA(dspy.Signature):
    """Answer questions about politics and history."""

    question = dspy.InputField(desc="a question about politics or history.")
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question="Who is the first president of United States of America?")
print(pred.answer)
pred = generate_answer(question="When did civil war happen?")
print(pred.answer)

George Washington.
1861-1865


# Few-Shot RAG

### Setup ColBERTv2

In [17]:
colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
# configure retrieval model
dspy.settings.configure(rm=colbertv2)

### Load Dataset

In [18]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=69, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

### Define Signature

In [19]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

### Build RAG pipeline

In [20]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)  # this should work since we've configured our retrieval model early on
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)  # use the signature we defined in the previous step
    
    def forward(self, question):
        # retrieve context = 3 passages from ColBERTv2
        context = self.retrieve(question).passages
        # Use CoT module to generate answer given a question
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

## Inference (uncompiled)

In [21]:
rag_qa = RAG()
# Ask any question you like to this simple RAG program.
my_question = "What castle did David Gregory inherit?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = rag_qa(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: What castle did David Gregory inherit?
Predicted Answer: Kinnairdy
Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 t...', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...']
